In [61]:
import argparse
import os
import sys

import h5py
from einops import rearrange

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
from configmypy import ConfigPipeline, YamlConfig
from torch import nn
from torch.nn.parallel import DistributedDataParallel as DDP

from scripts.baseline.utils import *
from src.data_loading import getTestData
from src.neuraloperators.neuralop import get_model
from src.neuraloperators.neuralop.training import setup

months  = [
    str(i).rjust(2,'0') for i in range(1,13,1)
]

data_name = 'era5'
data_path = './datasets/era5_new'
method = 'bicubic'
crop_size = 128
n_patches = 8
noise_ratio = 0.0

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 32
seed = 5544
upsampling_factor = 4
zero_shot = True
zero_shot_upsampling_factor = 16

project_dir = '/home/rpdemilt/climate_downscaling/neural-operators-weather-downscaling'

resol, n_fields, mean, std = get_data_info(data_name)
test1_loader, test2_loader = None, None

upsampling_factor = zero_shot_upsampling_factor

pipe = ConfigPipeline(
        [
            YamlConfig(
                "./config.yaml", config_name="default", config_folder=os.path.join(project_dir,'src/neuraloperators/config')
            ),
            YamlConfig(config_folder=os.path.join(project_dir,'src/neuraloperators/config')),
        ]
    )
config = pipe.read_conf()
arch = config["arch"].lower()
config_arch = config.get(arch)

# Set-up distributed communication, if using
device, is_logger = setup(config)

# Make sure we only print information when needed
config.verbose = config.verbose and is_logger

# Print config to screen
if config.verbose and is_logger:
    pipe.log()
    sys.stdout.flush()

model = get_model(config, mean, std, upsampling_factor)
model = model.to(device)

checkpoint_path = os.path.join(project_dir,'src/neuraloperators',config_arch["checkpoint_path"])
if config.checkpoint.save_checkpoint is True:
    model.load_state_dict(torch.load(os.path.join(checkpoint_path, 'best_model.pt')))

out_channels = config_arch["out_channels"]
model.eval()

###############################
#####    CONFIGURATION    #####
###############################

Steps:
------
 (1) YamlConfig with config_file=./config.yaml, config_name=default, config_folder=/home/rpdemilt/climate_downscaling/neural-operators-weather-downscaling/src/neuraloperators/config
 (2) YamlConfig with config_file=None, config_name=None, config_folder=/home/rpdemilt/climate_downscaling/neural-operators-weather-downscaling/src/neuraloperators/config

-------------------------------

Configuration:
--------------

n_params_baseline=None
verbose=True
arch=tfno2d
distributed.use_distributed=False
distributed.wireup_info=mpi
distributed.wireup_store=tcp
distributed.model_parallel_size=2
distributed.seed=5544
tfno2d.data_channels=3
tfno2d.out_channels=3
tfno2d.n_modes_height=8
tfno2d.n_modes_width=8
tfno2d.hidden_channels=32
tfno2d.lifting_channels=256
tfno2d.projection_channels=256
tfno2d.n_layers=4
tfno2d.domain_padding=None
tfno2d.domain_padding_mode=one-sided
tfno2d.fft_norm=fo

TFNO2d(
  (shiftmean_x): ShiftMean()
  (shiftmean_y): ShiftMean()
  (fno_blocks): LocalFNOBlocks(
    (convs): SpectralConv(
      (weight): ModuleList(
        (0-3): 4 x ComplexDenseTensor(shape=torch.Size([32, 32, 8, 5]), rank=None)
      )
    )
    (fno_skips): ModuleList(
      (0-3): 4 x Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
    (differential): ModuleList(
      (0-3): 4 x FiniteDifferenceConvolution(
        (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same, bias=False, padding_mode=circular)
      )
    )
    (mlp): ModuleList(
      (0-3): 4 x MLP(
        (fcs): ModuleList(
          (0): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
          (1): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1))
        )
      )
    )
    (mlp_skips): ModuleList(
      (0-3): 4 x SoftGating()
    )
    (norm): ModuleList(
      (0-7): 8 x GroupNorm(1, 32, eps=1e-05, affine=True)
    )
  )
  (lifting): MLP(
    (fcs): ModuleList(
     

In [62]:
months  = [
    str(i).rjust(2,'0') for i in range(1,13,1)
]

In [ ]:
climate_vars = ['t2m','u10','v10','tcw']

path = './datasets/era5_california/2022/01/data_stream-oper_stepType-instant.nc'

file = h5py.File(path)

arr = np.stack(file[key] for key in climate_vars)

arr = np.transpose(arr, (1,0,2,3))

spd = np.sqrt(arr[:,1,:,:]**2 + arr[:,2,:,:]**2)

arr = np.stack([arr[:,0,:,:],spd,arr[:,3,:,:]],axis=1)

hrs = 24
date_aggregate = rearrange(arr,'(f t) c h w -> f t c h w',t=hrs)
date_aggregate =np.mean(date_aggregate,axis=1)
print(date_aggregate.shape)

tensor = torch.from_numpy(arr)
tensor.shape

(31, 3, 41, 44)


torch.Size([744, 3, 41, 44])

In [64]:
data = []
for month in months:
    path = f'./datasets/era5_california/2022/{month}/data_stream-oper_stepType-instant.nc'

    file = h5py.File(path)

    arr = np.stack(file[key] for key in climate_vars)

    arr = np.transpose(arr, (1,0,2,3))

    spd = np.sqrt(arr[:,1,:,:]**2 + arr[:,2,:,:]**2)

    arr = np.stack([spd,arr[:,0,:,:],arr[:,3,:,:]],axis=1)

    hrs = 24
    date_aggregate = rearrange(arr,'(f t) c h w -> f t c h w',t=hrs)
    date_aggregate =np.mean(date_aggregate,axis=1)


    tensor = torch.from_numpy(date_aggregate)
    data.append(tensor)

data = torch.cat(data,dim=0)

In [65]:
data.shape

torch.Size([365, 3, 41, 44])

In [66]:
batch_size = 1
num_batches = len(data) // batch_size

downscaled_year = []

for i in range(num_batches):
    X = data[(i*batch_size):((i+1)*batch_size)].to(device)

    with torch.no_grad():
        X_hr = model(X)

    sample = X_hr.cpu().detach().numpy()
    downscaled_year.append(sample)

output = np.concatenate(downscaled_year,axis=0)

In [67]:
output.shape

np.save('california_2022_fno_16times_downscale.npy',output)